# Building Applications with LLMs

- [Skool Link](https://www.skool.com/data-alchemy/classroom/a455582e?md=cffccd9c21af41e0bc3669610ce3bc39)
- [YouTube](https://www.youtube.com/watch?time_continue=1783&v=NYSWn1ipbgg&embeds_referring_euri=https%3A%2F%2Fwww.skool.com%2F&embeds_referring_origin=https%3A%2F%2Fwww.skool.com&source_ve_path=Mjg2NjY&feature=emb_logo)
- [LangChain Experiments](https://github.com/daveebbelaar/langchain-experiments)

In [1]:
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)

True

In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from dotenv import find_dotenv, load_dotenv
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import textwrap

load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
def create_db_from_youtube_video_url(video_url):
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
    docs = text_splitter.split_documents(transcript)

    db = FAISS.from_documents(docs, embeddings)
    return db

In [4]:
def get_response_from_query(db, query, k=4):
    docs = db.similarity_search(query, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])

    chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.2)

    # Template to use for the system message prompt
    template = """
        You are a helpful assistant that that can answer questions about youtube videos 
        based on the video's transcript: {docs}
        
        Only use the factual information from the transcript to answer the question.
        
        If you feel like you don't have enough information to answer the question, say "I don't know".
        
        """

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    # Human question prompt
    human_template = "Answer the following question: {question}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    chain = LLMChain(llm=chat, prompt=chat_prompt)

    response = chain.run(question=query, docs=docs_page_content)
    response = response.replace("\n", "")
    return response, docs

In [5]:
# Example usage:
video_url = "https://www.youtube.com/watch?v=th4j9JxWGko"
db = create_db_from_youtube_video_url(video_url)

query = "what is this video about?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width=50))

/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


This video is about the presenter's new data
science workflow using AI-powered tools like Chat
GPT and GitHub co-pilot. They demonstrate how they
tackle a data science project using a new data set
and show their thought process and interaction
with these tools to predict measurements in a
multi-stage continuous flow manufacturing process.


In [6]:
video_url ="https://www.youtube.com/live/GgPGnJeQdLY?si=Atn8tfUJTC-QE-rv"

In [7]:
db = create_db_from_youtube_video_url(video_url)

query = "what is this video about?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width=50))

This video appears to be about the concept of
letting go of control and the negative effects of
constantly trying to control and manipulate our
lives. It discusses how our thoughts and ego can
keep us trapped in a virtual reality and prevent
us from experiencing true freedom and peace. The
video also shares stories and teachings from
various spiritual traditions to illustrate these
concepts.


In [8]:
query = "who is the speaker in this video?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width=50))

The speaker in this video is not mentioned in the
transcript.


In [9]:
query = "what does the speaker mention about creativity?"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response, width=50))

The speaker mentions that creativity is not hard
to see and that it is connected to feeling
present, feeling at home in oneself, and feeling
connected to others.
